In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("train_dataset_train.csv")

# Data preparation

## Step 1

In [ ]:
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes

In [ ]:
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes

In [ ]:
def op_t(x):
    return x[:x.find('_')]
def op_a(x):
    return x[x.find('_')+1:]

In [ ]:
df['op_t'] = df['oper_type + oper_attr'].apply(op_t).astype('int64')
df['op_a'] = df['oper_type + oper_attr'].apply(op_a).astype('int64')

In [ ]:
def p_to_int(x):
    if type(x) == float:
        return int(x)
    return x[:x.find('.')]

df['index_oper'] = df['index_oper'].apply(p_to_int)

In [ ]:

df['priority'] = df['priority'].apply(p_to_int)

In [ ]:
qq = pd.read_csv('PIndx20.csv')

In [ ]:
qq['index_oper'] = qq['INDEX'].values

In [ ]:
def nd(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 >0:
        return 6
    elif x // 10**4 >0:
        return 5
    elif x // 10**3 >0:
        return 4

def from5t6(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 == 0:
        return x*10
    return x



In [ ]:
pz = pd.merge(df[['index_oper','index_oper2']], qq[['index_oper2','ACTDATE']],how = 'left', on="index_oper2")
po = pd.merge(df[['index_oper','index_oper3']], qq[['index_oper3','ACTDATE']],how = 'left', on="index_oper3")

In [ ]:
pld2 = pd.DataFrame(columns = ['index_oper','new_index_oper','OPSSUBM','REGION','AUTONOM','CITY','ACTDATE'])
pld = pd.DataFrame(columns = ['index_oper','new_index_oper'])
pld['index_oper'] = df['index_oper'].values
pld2['index_oper'] = df['index_oper'].values
qq['new_index_oper'] = qq['INDEX'].values

for i in range(10):
    def from5t6(x):
        if x == '':
            return 0
        x = int(x)
        if x // 10**5 == 0:
            return x*10 + i
        return x
    pld['new_index_oper'] = pld['index_oper'].apply(from5t6)
    pld2['new_index_oper'] = pld2['index_oper'].apply(from5t6)
    
    pz = pd.merge(pld, qq[['new_index_oper','OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']],how = 'left', on="new_index_oper")
    
    pld['index_oper'][pz['ACTDATE'].isna() == False]['index_oper'] = pz[pz['ACTDATE'].isna() == False]['new_index_oper'].values
    pld2['index_oper'][pz['ACTDATE'].isna() == False]['index_oper'] = pz[pz['ACTDATE'].isna() == False]['new_index_oper'].values
    
    pld2.loc[pz['ACTDATE'].isna() == False,['OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']] = pz[pz['ACTDATE'].isna() == False][['OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']].values
    
    
    

    
    

In [ ]:
pld2[pld2['index_oper'] == 641769] = [[641769,641769,641760,'КУРГАНСКАЯ ОБЛАСТЬ','','ДАЛМАТОВО','2021-01-01'] for i in range(54)]
pld2[pld2['index_oper'] == '64142'] = [[641420,641420,641420,'КУРГАНСКАЯ ОБЛАСТЬ','','ЗВЕРИНОГОЛОВСКОЕ','2021-01-01'] for i in range(38)]
pld2[pld2['index_oper'] == 630301] = [[630300,630300,630300,'НОВОСИБИРСКАЯ ОБЛАСТЬ','','НОВОСИБИРСК','2021-01-01'] for i in range(36)]
pld2[pld2['index_oper'] == '62608'] = [[626080,626080,626080,'ТЮМЕНСКАЯ ОБЛАСТЬ','','ИСЕТСКОЕ','2021-01-01'] for i in range(30)]
pld2[pld2['index_oper'] == 626089] = [[626080,626080,626080,'ТЮМЕНСКАЯ ОБЛАСТЬ','','ИСЕТСКОЕ','2021-01-01'] for i in range(19)]
pld2[pld2['index_oper'] == 641429] = [[641420,641420,641420,'КУРГАНСКАЯ ОБЛАСТЬ','','ЗВЕРИНОГОЛОВСКОЕ','2021-01-01'] for i in range(17)]
pld2[pld2['index_oper'] == 356247] = [[356247,356247,356240,'СТАВРОПОЛЬСКИЙ КРАЙ','','МИХАЙЛОВСКОЕ','2021-01-01'] for i in range(2)]

df.drop(['index_oper2','index_oper3'],axis = 1,inplace = True)

save in ('big_df.csv')

## Step 2

In [ ]:
big_df = pd.read_csv('big_df.csv')

In [ ]:
big_df = big_df.drop(['Unnamed: 0'],axis = 1)

In [ ]:
big_df['net_weight'] = (big_df['weight'] - big_df['weight_mfi'])
big_df['total_price'] = (big_df['price_mfi'] + big_df['transport_pay'])

big_df['w/p'] = np.log(big_df['weight_mfi'] + 1) - np.log(big_df['price_mfi']+1)



In [ ]:
big_df['post_days'] = pd.to_datetime('2020-11-1') - pd.to_datetime(big_df['ACTDATE'])

In [ ]:
big_df['post_days'] = big_df['post_days'].apply(lambda x: x.days)

In [ ]:
big_df['delta_log1'] = np.log(big_df['total_qty_oper_login_1']+1) - np.log(big_df['dist_qty_oper_login_1']+1)

In [ ]:
big_df['delta_qty_log'] = (big_df['total_qty_oper_login_1'] - big_df['total_qty_oper_login_0'])

In [ ]:
big_df['delta_bigi'] = (big_df['total_qty_over_index'] - big_df['total_qty_over_index_and_type'])

In [ ]:
def num_tov(x):
    if x == '0' or x=='':
        return 0
    else:
        return x.count(',')+1
    

big_df['num_prod'] = big_df['name_mfi'].apply(num_tov)

In [ ]:
test_df = pd.read_csv('test_dataset_test.csv')

In [ ]:
qq['index_oper'] = qq['INDEX'].values
def p_to_int(x):
    if type(x) == float:
        return int(x)
    return x[:x.find('.')]


def nd(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 >0:
        return 6
    elif x // 10**4 >0:
        return 5
    elif x // 10**3 >0:
        return 4

def from5t6(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 == 0:
        return x*10
    return x


def nd(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 >0:
        return 6
    elif x // 10**4 >0:
        return 5
    elif x // 10**3 >0:
        return 4

def from5t6(x):
    if x == '':
        return 0
    x = int(x)
    if x // 10**5 == 0:
        return x*10
    return x


    
    
test_df['index_oper'] = test_df['index_oper'].apply(p_to_int)


pld2 = pd.DataFrame(columns = ['index_oper','new_index_oper','OPSSUBM','REGION','AUTONOM','CITY','ACTDATE'])
pld = pd.DataFrame(columns = ['index_oper','new_index_oper'])
pld['index_oper'] = test_df['index_oper'].values
pld2['index_oper'] = test_df['index_oper'].values
qq['new_index_oper'] = qq['INDEX'].values

for i in range(10):
    def from5t6(x):
        if x == '':
            return 0
        x = int(float(x))
        if x // 10**5 == 0:
            return x*10 + i
        return x
    pld['new_index_oper'] = pld['index_oper'].apply(from5t6)
    pld2['new_index_oper'] = pld2['index_oper'].apply(from5t6)
    
    pz = pd.merge(pld, qq[['new_index_oper','OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']],how = 'left', on="new_index_oper")
    
    pld['index_oper'][pz['ACTDATE'].isna() == False]['index_oper'] = pz[pz['ACTDATE'].isna() == False]['new_index_oper'].values
    pld2['index_oper'][pz['ACTDATE'].isna() == False]['index_oper'] = pz[pz['ACTDATE'].isna() == False]['new_index_oper'].values
    
    pld2.loc[pz['ACTDATE'].isna() == False,['OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']] = pz[pz['ACTDATE'].isna() == False][['OPSSUBM','REGION','AUTONOM','CITY','ACTDATE']].values


pld2['AREA'] = (pld2['REGION'].fillna('') + pld2['AUTONOM'].fillna('')).values
pld2.drop(['REGION','AUTONOM'],axis = 1,inplace = True)
pld2.loc[pld2['AREA'] == 'МОСКВА','CITY'] = pld2[pld2['AREA'] == 'МОСКВА']['CITY'].fillna('МОСКВА').values

big_test_df = pd.concat([test_df,pld2.drop(['index_oper'],axis=1)],axis = 1)
big_test_df['ACTDATE'] = pd.to_datetime(big_test_df['ACTDATE'])







In [ ]:
def npust(x):
    if x == '':
        return 0
    return int(x)
big_test_df['index_oper'] = big_test_df['index_oper'].apply(npust)
big_test_df['new_index_oper'] = big_test_df['new_index_oper'].apply(npust)
big_test_df['OPSSUBM'] = big_test_df['OPSSUBM'].fillna(0).apply(npust)

## Step 3

In [ ]:
a = big_df[['OPSSUBM']]

In [ ]:
zips = pd.read_csv('zip_to_lat_lon_Europe.csv')

In [ ]:
zips = zips[zips['Country'] == 'Russia']

In [ ]:
zips = zips[['postal code','latitude','longitude']]

In [ ]:
a['OPSSUBM'] = a['OPSSUBM'].astype('str')
b = big_df[['new_index_oper']]
b['new_index_oper'] = big_df['new_index_oper'].astype('str')

In [ ]:
zips.columns = ['OPSSUBM','latitude','longitude']

In [ ]:
wer['latitude'] =wer['latitude'].astype('float64')
wer['longitude'] =wer['longitude'].astype('float64')

In [ ]:
wer = pd.DataFrame([['102975', '102976', '420300', '108981', '630874', '200983', '620980','140988', '140960', '130210'],
[55.583276,55.583276,55.615,55.58,51.50,59.84,56.50,55.306,55.306,55.3],
[37.175359,37.175359,49.268,37.19,35.33,30.425,60.35,37.52,37.52,37.52]]).T

wer.columns = ['OPSSUBM','latitude','longitude']
zips2 = pd.concat([zips,wer],axis = 0,ignore_index = True)

In [ ]:
ab = a.merge(zips2,on = 'OPSSUBM',how = 'left')

In [ ]:
import geopy.distance
def dist_sha(x):
    if np.isnan(x['latitude']):
        return np.nan
    return geopy.distance.geodesic((x['latitude'],x['longitude']),(55.5832, 37.1753)).km

In [ ]:
zips2['distance'] = zips2.apply(dist_sha,axis = 1)

In [ ]:
distance_tr = ab.apply(dist_sha,axis = 1)

In [ ]:
a2 = big_df[['OPSSUBM']]
a2['OPSSUBM'] = a2['OPSSUBM'].astype('str')
ab2 = a2.merge(zips2,on = 'OPSSUBM',how = 'left')
distance_test = ab2.apply(dist_sha,axis = 1)

## Step 4

In [ ]:
big_test = pd.read_csv('last_big_test.csv').drop(['Unnamed: 0'],axis = 1)
big_train = pd.read_csv('last_big_train.csv').drop(['Unnamed: 0'],axis = 1)
text_features = pd.read_csv('new_text_features.csv').drop(['Unnamed: 0'],axis = 1) # Собирается в Pochta_text.ipynb

In [ ]:
gen_features = pd.read_csv('gen_features.csv') # Собирается в 'pochta6last.ipynb' 'pochta6generate.ipynb' 'pochta6generate+nlp.ipynb'  'pochta6last_last13.ipynb'

In [ ]:
gen_features = gen_features[['delta_qty_log.MODE(univer.type)','delta_bigi.COUNT(univer)','total_qty_over_index.COUNT(univer)','mailctg.MODE(univer.type)']]

In [ ]:
all_big = pd.DataFrame()

In [ ]:
cols = [ 'id', 'type', 'weight', 'mailctg', 'mailrank',
       'transport_pay', 'postmark', 'weight_mfi', 'price_mfi',
       'dist_qty_oper_login_1', 'total_qty_oper_login_1',
       'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       'total_qty_over_index', 'is_wrong_phone_number', 'op_t',
       'op_a', 'CITY', 'AREA', 'net_weight', 'total_price', 'rel_price', 'w/p',
       'w*p', 'post_days', 'delta_log1', 'delta_qty_log', 'delta_bigi',
       'num_prod', 'cluster', 'mail%0', 'mail%1', 'mail%-1']

In [ ]:
big_train = big_train[cols]
big_test = big_test[cols]

In [ ]:
all_big = pd.concat([big_train,big_test],ignore_index = True)

In [ ]:
all_big

In [ ]:
all_big = pd.concat([all_big,pd.concat([text_features.drop(['text'],axis = 1),gen_features],axis = 1)],axis = 1)

In [ ]:
all_big.to_csv('all_big.csv') # cluster собирается в pochta2t{i}.ipynb i - [1,10] и pochta3(words).ipynb

In [ ]:
tt_dis = pd.concat([big_df[['id','OPSSUBM']],big_test_df[['id','OPSSUBM']]],ignore_index = True)

In [ ]:
tt_dis['OPSSUBM'] = tt_dis['OPSSUBM'].astype('str')

In [ ]:
tt = tt_dis.merge(zips2[['OPSSUBM','distance']],how='left',on='OPSSUBM')

In [ ]:
tt.to_csv('/Users/kirillgolubev/PycharmProjects/Pochta/didi.csv')

In [ ]:
%%capture
!pip install lightautoml

In [ ]:
!pip install gdown

Сборку приведенных ниже наборов данных можно найти в Auxiliary notebooks

In [ ]:
!gdown --id 1luhGPR0zWXCmVAsa5tA74e5z9FE63y3F

In [ ]:
!gdown --id 1mM1Jt3HcwfEx6lVsqbwHiVFQJuW28WV6

In [ ]:
!gdown --id 1qko7C-EEk1Smf9dB46iSc7DU5kEKIp87

In [ ]:
!gdown --id 14mCgEKtF_Z-p6qLkvs-ovM1WtilHy_N-

In [ ]:
!gdown --id 1zJeYlt2euRSJi_C25BGyu5LmO1pxPwJt

In [ ]:
!gdown --id 1nUNXbmncyEaimwlhOXpIlhsh8f-MLoEd

# Training model

In [ ]:

# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task


In [ ]:
gens = pd.read_csv('../input/end-gen/didi.csv')
for i in range(1,7):
    ptr = pd.read_csv(f'../input/end-gen/new_gener_{i}.csv')
    gens = pd.concat([gens,ptr],axis = 1)

In [ ]:
all_df = pd.read_csv('./all_big.csv')
label = pd.read_csv('./label.csv')
all_df.drop(['mobile_f', 'AREA', 'post_days', 'num_prod', 'mail%0', 'mail%-1', 'tech_f', 'accses_f', 'man_f',
       'cloth_f', 'child_f', 'is_0_mfi','Unnamed: 0'],axis = 1,inplace = True)

In [ ]:
all_df = pd.concat([all_df,gens.drop(['Unnamed: 0','id','OPSSUBM'],axis = 1)],axis = 1)

In [ ]:
for col in ['CITY','type']:
    for feat in ['net_weight','weight_mfi','transport_pay','total_price','weight','delta_qty_log','w/p']:
        mean = all_df[[col]].merge(all_df[[col,feat]].groupby(col).aggregate('mean'), how = 'left',on=col).fillna(0)[feat]
        std = all_df[[col]].merge(all_df[[col,feat]].groupby(col).aggregate('std'), how = 'left',on=col).fillna(1)[feat]
        all_df[f'{feat}_{col}'] = (all_df[feat] - mean)/std
        print(col,feat)

In [ ]:
all_df.drop(['op_t','op_a'],axis = 1,inplace = True)

In [ ]:
qty_cols = ['total_qty_over_index','total_qty_over_index_and_type','total_qty_oper_login_0','total_qty_oper_login_1','dist_qty_oper_login_1']
for i in range(5):
    for j in range(i+1,5):
        if (not (i==3 and j==4)) and (not (i ==2  and j==3)) and (not (i == 0 and j == 1)):
            all_df[f'delta_{i}_{j}'] =  (all_df[qty_cols[i]] - all_df[qty_cols[j]])
            all_df[f'div{i}_{j}'] =  np.log(all_df[qty_cols[i]]+1) - np.log(all_df[qty_cols[j]]+1)
            print(i,j)

In [ ]:

import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

def make_bucket(df,feature, n =  100):
    '''функция, бьющая на бакеты(по умолчанию 100 точек)'''
    return df.assign(bucket = np.ceil(df[feature].rank(pct = True) * n))

from sklearn.metrics import roc_auc_score,r2_score
from sklearn.linear_model import LogisticRegression

def fit_predict1(X_train, y_train, X_test, global_woe):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    prediction = model.predict_proba(X_train)
    return prediction, model.intercept_, model.coef_


def woe_val(tar, global_woe, iv, g_t, g_f):
    woe = np.log((sum(tar)) / (tar.shape[0] - sum(tar) + 1)) - global_woe
    iv += woe * ((sum(tar) / g_t) - (tar.shape[0] - sum(tar)) / g_f)
    return woe, iv


def woe_line(x, y, n_buckets):  #mid how target feature_name, target_name
    """ Строит график зависимости WoE
    x - параметр, от которого стоит искать зависимость
    y - метки класса (0 / 1)
    n_buckets - количество бинов для вещественного признака
    feature_name, target_name - подписи к графику
    """
    ind = np.argsort(x.reshape(-1))
    size = len(x) // n_buckets
    grouped = []
    med_var = []
    mini = []
    maxi = []
    woe = []
    iv = 0
    df = pd.DataFrame()
    df['tar'] = y
    df['var'] = x
    df = make_bucket(df, 'var', n_buckets)

    g_t = sum(y)
    g_f = y.shape[0] - g_t

    new_nbins = df['bucket'].nunique()
    
    global_woe = np.log(
        (sum(df['tar'].values)) / (df['tar'].shape[0] - sum(df['tar'].values) + 1))
    
    bucks = np.sort(df['bucket'].unique())
    

    for t in bucks:
        mini.append(np.min(df['var'][df['bucket'] == t]))
        maxi.append(np.max(df['var'][df['bucket'] == t]))
        med_var.append(np.median(df['var'][df['bucket'] == t]))
        w, iv = woe_val(df['tar'][df['bucket'] == t].values, global_woe,
                        iv, g_t, g_f)
        woe.append(w)
        df['bucket'] = df['bucket'].replace(
            t, f"{round(mini[-1],3)}...{round(maxi[-1])}")

    alpha = (med_var[-1] - med_var[0]) / 100

    pred, inter, coef = fit_predict1(
        x, y,
        np.array([med_var[0] - 5 * alpha] + med_var +
                 [med_var[-1] + 5 * alpha]).reshape(-1, 1), global_woe)

    plt.figure(figsize=(15, 7))

    plt.grid()
    plt.plot(med_var, woe, 'o', color='red')
    line_tar = plt.plot(med_var, woe, color='red', linestyle='--', label="Woe")

    log_reg1 = plt.plot([med_var[0], med_var[-1]], [
        med_var[0] * coef[0] + inter - global_woe,
        med_var[-1] * coef[0] + inter - global_woe
    ],
                        label="Interpolation")

    plt.legend(fontsize=12,
               loc='lower right',
               fancybox=True,
               framealpha=1,
               shadow=True)
    xmin, xmax = plt.xlim()
    ymin, ymax = plt.ylim()

    print("IV : ", round(iv, 4))
    print("AUC : ", round(roc_auc_score(y, pred[:, 1]), 4))
    print("n_buckets :", n_buckets)
    print("The point where woe is zero : ",
          round(((global_woe - inter) / coef[0])[0], 4))
    print("R_sqr : ", round(r2_score(woe,list(map(lambda x: coef[0]*x + inter - global_woe, med_var))),4))
    
    plt.xlabel("Feature", fontsize=20)
    plt.ylabel("WoE", fontsize=20)
    plt.grid()
    plt.show()

    fig = px.histogram(df, x='var', y='tar', color='bucket')
    fig.show()

def lo_sb(x):
    return np.sign(x)*np.log(abs(x)+1)


In [ ]:
all_df['w/p_CITY'] = (all_df['w/p_CITY'].clip(lower=-3, upper=3))
all_df['mailctg'] = all_df['mailctg'].astype('str')
all_df['delta_bigi.COUNT(univer)'] = np.log(all_df['delta_bigi.COUNT(univer)'])
all_df['weight'] = np.log(all_df['weight']+1)
all_df['delta_0_3'] = np.log(all_df['delta_0_3']+1)
all_df['delta_qty_log'] = lo_sb(all_df['delta_qty_log'].clip(-5216249,6753334))
all_df['total_qty_over_index.COUNT(univer)'] = np.log(all_df['total_qty_over_index.COUNT(univer)'].clip(1,500000))
all_df['delta_qty_log'] = all_df['delta_qty_log'].clip(-100,1000)
all_df['net_weight'] = np.abs(lo_sb(all_df['net_weight'].clip(-100,1000))).values
all_df['total_price_CITY'] = all_df['total_price_CITY'].fillna(0).clip(-1,1)
all_df['div0_4'] = all_df['div0_4'].clip(0,12)
all_df['net_weight_CITY'] = all_df['net_weight_CITY'].fillna(0).clip(-5,5)
all_df['delta_1_2'] = make_bucket(all_df[['delta_1_2']],'delta_1_2')['bucket']
all_df['delta_0_2'] = make_bucket(all_df[['delta_0_2']],'delta_0_2')['bucket']
all_df['total_price_type'] = all_df['total_price_type'].clip(-2,2)
all_df['price_mfi'] = all_df['price_mfi'].clip(0,1500)
all_df['weight_type'] = np.sqrt(all_df['weight_type'].fillna(0).clip(-0.7,2)+0.7)
all_df['cluster'] = all_df['cluster'].astype('str')
all_df['div1_4'] = all_df['div1_4'].fillna(0).clip(5,11)
all_df['transport_pay_CITY'] = all_df['transport_pay_CITY'].fillna(0).clip(-1,3)
all_df['transport_pay_type'] = all_df['transport_pay_type'].fillna(0).clip(-0.5,4)
all_df['delta_qty_log_type'] = all_df['delta_qty_log_type'].fillna(0).clip(-2,2)
all_df['net_weight_type'] = all_df['net_weight_type'].fillna(0).clip(-2,2)
all_df['weight_mfi_CITY'] = all_df['weight_mfi_CITY'].fillna(0).clip(-1,3)
all_df['w/p_type'] = all_df['w/p_type'].fillna(0).clip(-3,3)
all_df['weight_CITY'] = np.log(all_df['weight_CITY'].fillna(0).clip(-1,2) +2)
all_df['w/p'] = all_df['w/p'].fillna(0).clip(-3,2)
all_df['div0_3'] = all_df['div0_3'].fillna(0).clip(0,4)
all_df['delta_qty_log_CITY'] = all_df['delta_qty_log_CITY'].fillna(0).clip(-3,3)



In [ ]:
bum = ['weight', 'mailrank', 'transport_pay', 'postmark', 'weight_mfi',
       'price_mfi', 'dist_qty_oper_login_1', 'total_qty_oper_login_1',
       'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       'total_qty_over_index', 'is_wrong_phone_number', 'net_weight',
       'total_price', 'rel_price', 'w/p', 'w*p', 'delta_log1', 'delta_qty_log',
       'delta_bigi', 'mail%1', 'delta_bigi.COUNT(univer)',
       'total_qty_over_index.COUNT(univer)', 'distance',
       'CITY.MEAN(univer.weight_type)', 'CITY.STD(univer.delta_1_2)',
       'net_weight.MAX(univer.w*p)', 'rel_price.SKEW(univer.weight_type)',
       'rel_price.STD(univer.weight_type)', 'w*p.SKEW(univer.weight_type)',
       'net_weight_CITY', 'weight_mfi_CITY', 'transport_pay_CITY',
       'total_price_CITY', 'weight_CITY', 'delta_qty_log_CITY', 'w/p_CITY',
       'net_weight_type', 'weight_mfi_type', 'transport_pay_type',
       'total_price_type', 'weight_type', 'delta_qty_log_type', 'w/p_type',
       'delta_0_2', 'div0_2', 'delta_0_3', 'div0_3', 'delta_0_4', 'div0_4',
       'delta_1_2', 'div1_2', 'delta_1_3', 'div1_3', 'delta_1_4', 'div1_4',
       'delta_2_4', 'div2_4']
for col in bum:
    all_df[col] = all_df[col].fillna(0).clip(np.percentile(all_df[col].fillna(0).values, 1),np.percentile(all_df[col].fillna(0).values, 99))

In [ ]:
all_df['dist/price'] = np.log(all_df['distance']+1) - np.log(all_df['total_price']+1)
all_df['dist/weight'] = np.log(all_df['distance']+1) - np.log(all_df['weight']+1)

all_df['dist*price_log'] = np.log(all_df['distance']*all_df['total_price']+1)
all_df['dist*weight'] = all_df['distance']*all_df['weight']

In [ ]:
bum1 = ['dist/price', 'dist/weight', 'dist*price_log']
for col in bum1:
    all_df[col] = all_df[col].fillna(0).clip(np.percentile(all_df[col].fillna(0).values, 1),np.percentile(all_df[col].fillna(0).values, 99))

In [ ]:
big_train = all_df[:6000000]
big_test = all_df[6000000:]

In [ ]:
N_THREADS = 32
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 900*4*2 # equal to 15*4 minutes
TARGET_NAME = 'label'

In [ ]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [ ]:

tr_data, te_data = train_test_split(
    big_train, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

print(f'Data splitted. Parts sizes: tr_data = {tr_data.shape}, te_data = {te_data.shape}')

tr_data.head()

In [ ]:
cols = ['w/p', 'delta_qty_log.MODE(univer.type)', 'w*p',
       'rel_price.SKEW(univer.weight_type)', 'mailctg', 'weight_type',
       'weight_mfi_type', 'w*p.SKEW(univer.weight_type)', 'dist/price',
       'type', 'net_weight_type', 'div0_2', 'delta_bigi.COUNT(univer)',
       'total_qty_over_index.COUNT(univer)', 'delta_0_3', 'w/p_CITY',
       'delta_qty_log', 'weight_mfi_CITY', 'rel_price',
       'delta_qty_log_type', 'delta_1_2', 'net_weight.MAX(univer.w*p)',
       'net_weight_CITY', 'rel_price.STD(univer.weight_type)',
       'weight_mfi', 'is_wrong_phone_number', 'total_price_CITY',
       'total_price_type', 'delta_bigi', 'transport_pay_CITY', 'cluster',
       'weight_CITY', 'price_mfi', 'net_weight', 'delta_0_2',
       'dist/weight', 'div0_3', 'transport_pay_type', 'CITY',
       'dist*price_log', 'div0_4', 'dist_qty_oper_login_1', 'delta_log1',
       'div1_4', 'CITY.STD(univer.delta_1_2)', 'mail%1',
       'mailctg.MODE(univer.type)', 'weight', 'delta_1_3', 'div1_2',
       'delta_qty_log_CITY', 'div2_4', 'div1_3', 'dist*weight',
       'transport_pay', 'w/p_type', 'distance',
       'CITY.MEAN(univer.weight_type)', 'delta_1_4', 'total_price','id','label']

In [ ]:
task = Task('binary',loss = 'logloss',metric = 'logloss') #task = Task('binary', loss = 'logloss', metric = 'auc')

roles = {
    'target': TARGET_NAME,
    'drop': ['id']
}

automl = TabularAutoML(
    task = task, 
    timeout = 3600*6,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE})
    

In [ ]:
%%time 
oof_pred = automl.fit_predict(big_train, roles = roles,verbose=2)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [ ]:
import joblib

joblib.dump(automl, 'model_end.pkl')

# Submit

In [ ]:
automl = joblib.load('model_end.pkl')

In [ ]:
end_test_pred = automl.predict(big_test)

In [ ]:
submission = pd.DataFrame(columns = ['id','label'])
submission['id'] = big_test['id'].values
submission['label'] = 1*(end_test_pred.data[:,0] >= 0.01)

In [ ]:

submission.to_csv('submission_17_galactica_001.csv.csv',index = False)